<a href="https://colab.research.google.com/github/Sminjoo/Project_ORI-Team_movie_review/blob/main/modeling/SMJ_IMDB_word2vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd

# Attempt to read the file, handling potential errors
try:
    data = pd.read_csv('/content/drive/MyDrive/영화리뷰/IMDB Dataset.csv')
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")

    # Inspect the problematic row and surrounding rows
    with open('/content/drive/MyDrive/영화리뷰/IMDB Dataset.csv', 'r') as file:
        for i, line in enumerate(file):
            if i >= 11037 and i <= 11041:  # Check a few lines before and after
                print(f"Line {i}: {line.strip()}")

In [ ]:
duplicate_rows = data[data.duplicated()]
print("중복된 값:\n", duplicate_rows)

중복된 값:
                                                   review sentiment
3537   Quite what the producers of this appalling ada...  negative
3769   My favourite police series of all time turns t...  positive
4391   Beautiful film, pure Cassavetes style. Gena Ro...  positive
6352   If you liked the Grinch movie... go watch that...  negative
6479   I want very much to believe that the above quo...  negative
...                                                  ...       ...
49912  This is an incredible piece of drama and power...  positive
49950  This was a very brief episode that appeared in...  negative
49984  Hello it is I Derrick Cannon and I welcome you...  negative
49986  This movie is a disgrace to the Major League F...  negative
49991  Les Visiteurs, the first movie about the medie...  negative

[418 rows x 2 columns]


In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(0.7,0.5))
# sns.set(style="darkgrid")

# fig, axs = plt.subplots(1, 2, figsize=(14,7), sharey=True)

# # 긍정적인 리뷰의 히스토그램
# sns.histplot(
#     x=data[data['sentiment'] == 'positive']['review'].str.len(),
#     color='lightgreen',
#     bins=30,
#     edgecolor='black',
#     kde=True,  # 밀도 추정선을 추가
#     ax=axs[0]
# )

# axs[0].set_title("Positive Reviews", fontsize=16, fontweight='bold')
# axs[0].set_xlabel("Review Length (Characters)", fontsize=14, fontweight='bold')
# axs[0].set_ylabel("Frequency", fontsize=14, fontweight='bold')

# # 부정적인 리뷰의 히스토그램
# sns.histplot(
#     x=data[data['sentiment'] == 'negative']['review'].str.len(),
#     color='salmon',
#     bins=30,
#     edgecolor='black',
#     kde=True,  # 밀도 추정선을 추가
#     ax=axs[1]
# )

# axs[1].set_title("Negative Reviews", fontsize=16, fontweight='bold')
# axs[1].set_xlabel("Length (Characters)", fontsize=14, fontweight='bold')
# axs[1].set_ylabel("Frequency", fontsize=14, fontweight='bold')

# plt.tight_layout()
# plt.show()


In [ ]:
data['sentiment']= LabelEncoder().fit_transform(data['sentiment']) #'positive'는 1로, 'negative'는 0으로 변환 - 매핑해도됨

In [ ]:
# 리뷰를 소문자로 변환
data['review'] = data['review'].str.lower()

data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. <br /><br />the...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [ ]:
import re

change = {
    r"\'m": " am",
    r"\'s": " is",
    r"n\'t": " not",
    r"\'re": " are",
    r"\'ve": " have",
    r"\'ll": " will",
    r"\'d": " had"

}

#text 컬럼에서 대체 작업 수행
def replace_contractions(text):
    for pattern, replacement in change.items():
        text = re.sub(pattern, replacement, text)
    return text

    # 리뷰에 적용
data['review'] = data['review'].apply(replace_contractions)

data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. <br /><br />the...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there is a family where a little boy...,0
4,"petter mattei is ""love in the time of money"" i...",1


In [ ]:
data['review'].iloc[4005]

'this film captured my heart from the very beginning, when hearing quincy jones\' first notes or seeing the wonderful color of purple of the flowers in the meadows. this is truly a film to cry and die for...! the whole cast gives the best performance in a film i have seen in years and spielberg has really outdone himself! whoppi goldberg, margaret avery, oprah winfrey(oh lord!), danny glover, and the others, all give us their best and you can feel it - almost touch it! goldberg is celie, she gives her that insecurity and feeling of inferiority that is needed for the character, and we grow with her, we grow strong together with her, throughout the movie, and we triumph with her. margaret avery is wonderful as shug avery, even when she is at her most arrogant, and shows us that "sinners", indeed, "have souls too". the always sympathetic, charming danny glover makes a marvellous job at making people hate him and the magnificent music of(i had say sir)quincy jones adds even more beauty to 

In [ ]:
from bs4 import BeautifulSoup

# BeautifulSoup을 사용하여 HTML 태그 제거
#HTML 태그란 웹 페이지의 구조를 정의하고 내용을 표시하는 데 사용되는 마크업 언어의 요소
#보통 <>로 이루어져 있음

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
## text에서 대괄호 [] 사이에 있는 모든 텍스트를 제거
def remove_between_square_and_parentheses(text):
    # 대괄호 [] 안의 텍스트 제거
    text = re.sub(r'\[[^\]]*\]', '', text)
    # 소괄호 () 안의 텍스트 제거
    text = re.sub(r'\([^)]*\)', '', text)
    return text


#Removing the noisy text
## 텍스트에서 HTML 태그와 대괄호 안의 노이즈 텍스트를 제거
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_and_parentheses(text)
    return text

#Apply function on review column
## 각 리뷰에서 HTML 태그와 대괄호 안의 노이즈 텍스트를 제거
data['review']=data['review'].apply(denoise_text)

data.head()
#결론 html 태그 제거, 소괄호, 대괄호 안에 있는 텍스트 제거
#노이즈 텍스트 방법이 URL 제거, 이메일 주소 제거, 이모지 제거, 특수 문자 제거, 여러 공백 처리가 있는 것임

<ipython-input-12-bbb662afaa44>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there is a family where a little boy...,0
4,"petter mattei is ""love in the time of money"" i...",1


In [ ]:
chat_words = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [ ]:
# 줄임말이 포함된 리뷰를 찾는 함수
def contains_abbreviation(text):
    words = re.findall(r'\b\w+\b|[^\w\s]', text)
    for w in words:
        if w.lower() in chat_words:
            return True
    return False

# 줄임말이 포함된 리뷰 필터링
abbreviations_in_reviews = data[data['review'].apply(contains_abbreviation)]

# 결과 출력
print(abbreviations_in_reviews)

                                                  review  sentiment
1      a wonderful little production. the filming tec...          1
4      petter mattei is "love in the time of money" i...          1
12     so im not a big fan of boll is work but then a...          0
18     i remember this film,it was the first film i h...          1
35     i bought this film at blockbuster for $3.00, b...          0
...                                                  ...        ...
49955  i have not seen this movie, but i must. having...          1
49959  my thoughts on the movie, 9it was not good, no...          0
49968  i must say as a girl with a cowboy of my own,i...          1
49976  i saw the movie in the theater at its release,...          1
49993  robert colomb has two full-time jobs. he is kn...          0

[7239 rows x 2 columns]


In [ ]:
# 영문 알파벳과 공백을 제외한 모든 문자를 제거
data['review'] = data['review'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there is a family where a little boy...,0
4,petter mattei is love in the time of money is ...,1


In [ ]:
import re

def chat_conversion(text):
    new_text = []
    # 단어와 구두점을 분리하는 정규식
    words = re.findall(r'\b\w+\b|[^\w\s]', text)

    for w in words:
        lower_w = w.lower()  # 소문자로 변환하여 딕셔너리에서 찾음
        if lower_w in chat_words:
            new_text.append(chat_words[lower_w])
        else:
            new_text.append(w)

    return " ".join(new_text)

data['review'] = data['review'].apply(chat_conversion)

data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there is a family where a little boy...,0
4,petter mattei is love in the time of money is ...,1


In [ ]:
data['review'].iloc[4]

'petter mattei is love in the time of money is a visually stunning film to watch mister mattei offers us a vivid portrait about human relations this is a movie that seems to be telling us what money power and success do to people in the different situations we encounter this being a variation on the arthur schnitzler is play about the same theme the director transfers the action to the present time new york where all these different characters meet and connect each one is connected in one way or another to the next person but no one seems to know the previous point of contact stylishly the film has a sophisticated luxurious look we are taken to see how these people live and the world they live in their own habitatthe only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits a big city is not exactly the best place in which human relations find sincere fulfillment as one discerns is the case with most of the people we encounterthe 

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# punkt 토크나이저 모델 다운로드
nltk.download('punkt')

def tokenize_text(text):
    # 텍스트를 단어 단위로 토큰화
    tokens = word_tokenize(text)
    return tokens

# 정제된 텍스트에 대해 토큰화 적용
data['review'] = data['review'].apply(tokenize_text)

# 변환된 데이터프레임의 상위 5개 행을 표시
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,review,sentiment
0,"[one, of, the, other, reviewers, has, mentione...",1
1,"[a, wonderful, little, production, the, filmin...",1
2,"[i, thought, this, was, a, wonderful, way, to,...",1
3,"[basically, there, is, a, family, where, a, li...",0
4,"[petter, mattei, is, love, in, the, time, of, ...",1


In [ ]:
import nltk

from nltk.corpus import stopwords

# stopwords 데이터 다운로드
nltk.download('stopwords')

# 불용어 집합 정의
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    # 불용어를 토큰 리스트에서 제거
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

# 토큰화된 텍스트에서 불용어 제거 적용
data['review'] = data['review'].apply(remove_stopwords)

# 변환된 데이터프레임의 상위 5개 행을 표시
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,review,sentiment
0,"[one, reviewers, mentioned, watching, oz, epis...",1
1,"[wonderful, little, production, filming, techn...",1
2,"[thought, wonderful, way, spend, time, hot, su...",1
3,"[basically, family, little, boy, thinks, zombi...",0
4,"[petter, mattei, love, time, money, visually, ...",1


In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop('sentiment', axis=1)
y = data['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 워드 임베딩

In [ ]:
from collections import Counter
from nltk import FreqDist
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from gensim.models import Word2Vec

# Word2Vec 모델 학습
w2v_model = Word2Vec(sentences=data['review'], vector_size=100, window=5, min_count=1, workers=4, sg=0)

# 학습된 단어 임베딩 벡터 확인
word_embedding = w2v_model.wv['example']  # 'example' 단어의 벡터
print("Word2Vec embedding for 'example':", word_embedding)

Word2Vec embedding for 'example': [ 0.56830007 -0.05672435  0.82880217 -0.64489156  1.0935516  -0.9355001
 -0.9291721   2.2622683  -0.37984136 -1.9137414  -0.11256565 -3.0366523
  0.19367798 -0.04548172  0.64039415 -0.28204077 -1.5696206  -0.46524572
  0.35340923  0.17623088 -0.49962816 -0.19584605 -1.4508959  -0.71826875
  0.476891    1.112031    0.567174   -0.57303685 -1.4392384  -0.0709766
  1.3381071   0.02300297 -0.95724565 -0.22959308 -0.7529377   1.7004757
  0.86976033 -0.10105044 -0.40389     1.4393793  -0.44975904  0.88106793
  1.381102   -0.890698   -1.2443144   0.01852879 -0.47039402 -0.48422354
 -0.20646614 -2.689869    0.8303824  -0.37547526 -1.116592    0.24873492
 -1.1884726   0.6581519   0.50606143  0.5386125  -1.0192987  -0.55925196
  0.59820014  1.8116294   0.91432345 -0.5881291  -0.11856946  0.78680235
 -0.7380839   0.29340035 -1.480404   -1.5144051  -0.92323285  0.04643961
 -0.6545389   0.43710122  0.1188904   0.07740234  0.45383853  0.11948156
 -1.0713044  -0.24339

In [ ]:
X_train = X_train['review'].tolist()

# 텍스트 데이터를 시퀀스로 변환
word_tokenizer  = Tokenizer()
word_tokenizer.fit_on_texts(X_train)

X_train_sequences = word_tokenizer.texts_to_sequences(X_train)
X_test_sequences = word_tokenizer.texts_to_sequences(X_test)

In [ ]:
# 패딩 처리 (모델에 고정된 입력 크기를 맞추기 위해)
max_length = max(len(seq) for seq in X_train_sequences)  # 최대로 긴 시퀀스를 기준으로 패딩
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')
vocab_length = len(word_tokenizer.word_index) + 1 # Tokenizer가 만든 단어 사전 크기 + 패딩을 위한 0번 인덱스

In [ ]:
print(f"Vocabulary Length: {vocab_length}")
print(f"X_train_padded Shape: {X_train_padded.shape}")
print(f"X_test_padded Shape: {X_test_padded.shape}")

Vocabulary Length: 170307
X_train_padded Shape: (39665, 1098)
X_test_padded Shape: (1, 1098)


In [ ]:
# 학습 및 테스트 데이터 분리
X_train_padded, X_test_padded, y_train, y_test = train_test_split(X_train_padded, y_train, test_size=0.2, random_state=42)

In [ ]:
print(f"X_train_padded Shape: {X_train_padded.shape}")
print(f"y_train Shape: {y_train.shape}")
print(f"X_test_padded Shape: {X_test_padded.shape}")
print(f"y_test Shape: {y_test.shape}")

X_train_padded Shape: (31732, 1098)
y_train Shape: (31732,)
X_test_padded Shape: (7933, 1098)
y_test Shape: (7933,)


In [ ]:
# 6. Word2Vec 임베딩 행렬 생성
#vocab_length = len(word_tokenizer.word_index) + 1 # 위에서 지정해줌
embedding_dim = 100

embedding_matrix = np.zeros((vocab_length, embedding_dim))
for word, i in word_tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    else:
        embedding_matrix[i] = np.random.normal(size=(embedding_dim,))  # 임베딩이 없는 단어는 무작위로 초기화

In [ ]:
# LSTM 모델 정의

max_length = 1429

model = Sequential([
    Embedding(input_dim=vocab_length, output_dim=embedding_dim, input_length=max_length,
    weights=[embedding_matrix], trainable=False),  # 사전 학습된 가중치 사용, 고정
    LSTM(64, return_sequences=True),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Flatten(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

In [ ]:
# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# 모델 학습
history = model.fit(X_train_padded, y_train, epochs=30, batch_size=200, validation_data=(X_test_padded, y_test),
                    callbacks=[early_stopping])

Epoch 1/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 450s 3s/step - accuracy: 0.7634 - loss: 0.4696 - val_accuracy: 0.8763 - val_loss: 0.3049
Epoch 2/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 498s 3s/step - accuracy: 0.8853 - loss: 0.2826 - val_accuracy: 0.8823 - val_loss: 0.2851
Epoch 3/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 503s 3s/step - accuracy: 0.8920 - loss: 0.2635 - val_accuracy: 0.8761 - val_loss: 0.2858
Epoch 4/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 503s 3s/step - accuracy: 0.9013 - loss: 0.2477 - val_accuracy: 0.8785 - val_loss: 0.2907
Epoch 5/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 500s 3s/step - accuracy: 0.9074 - loss: 0.2318 - val_accuracy: 0.8797 - val_loss: 0.2818
Epoch 6/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 501s 3s/step - accuracy: 0.9129 - loss: 0.2161 - val_accuracy: 0.8840 - val_loss: 0.2780
Epoch 7/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 448s 3s/step - accuracy: 0.9225 - loss: 0.1962 - val_accuracy: 0.8847 - val_loss: 0.2820
Epoch 8/30
159/159 ━━━━━━━━━━━━━━━━━━━━ 503s 3s/step - accuracy: 0.9226 - loss: 0.1895 - val_accu

In [ ]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

248/248 ━━━━━━━━━━━━━━━━━━━━ 41s 167ms/step - accuracy: 0.8862 - loss: 0.2746
Test Accuracy: 0.8840
Test Loss: 0.2780
